<a href="https://colab.research.google.com/github/akamojo/PAD-project/blob/master/part2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PAD 2nd Assignment PART 2
- Mitchell Galvao MIEI 41646
- Urszula Walińska MIEI 56556

## Automatic Extraction of Explicit and Implicit Keyword  
In our method, we first extract Relevant Expressions from a set of several documents, using 
LocalMaxs  extractor  we  have  implemented before. In our solution, we use TF-IDF measure as a criteria to select the most informative RE.

In [0]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

     | 768kB 1.2MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-evzohiv4/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
import re
from tqdm import tqdm_notebook as tqdm

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
file = open("/content/drive/My Drive/STUDIA/SEM 8/pad/en1.8m.txt", "r", encoding="utf8")
# file = open("en1.8m.txt", "r", encoding="utf8")

whole_text = file.read()
text = whole_text.split('\n')

In [0]:
def append_space(text, pattern, left=True):
    p = re.compile("[" + pattern + "]")
    
    for i, m in enumerate(p.finditer(text)):
        if left:
            text = text[: m.start() + i] + " " + text[m.start() + i :]
        else:
            text = text[: m.start() + 1 + i] + " " + text[m.start() + 1 + i :]
            
    return text

In [0]:
def preprocessing(text):
    for i in range(len(text)):
        text[i] = append_space(text[i], ",\])>")
        text[i] = append_space(text[i], ".")
        text[i] = append_space(text[i], "\[(<", False)
      
    return text

In [0]:
def build_n_gram(text_arr, n_gram, grams):
    cur_gram = []
    count_gram = 0

    for i in range(len(text_arr)):

        cur_gram = [text_arr[i]]

        for j in range(i + 1, i + n_gram):
            if j < len(text_arr):
                cur_gram.append(text_arr[j])

        if len(cur_gram) == n_gram:
            try:
                grams[" ".join(cur_gram)] += 1
            except KeyError:
                grams[" ".join(cur_gram)] = 1

    return grams

In [0]:
def build_dict(text, n_s):
    dictionary = []
    for n in range(n_s):
        dictionary.append(dict())
    
    for n in range(n_s):
        for line in text:
            dictionary[n] = build_n_gram(line.split(), n + 1, dictionary[n])
        
    return dictionary

In [0]:
def SCP_f(words, dictionary):
    words_arr = words.split()
    
    numerator = dictionary[len(words_arr) - 1][words] ** 2
    F = 1 / (len(words_arr) - 1)
    denominator = 0
    
    for i in range(len(words_arr) - 1):
        left = words_arr[0:i+1]
        right = words_arr[i+1:]
        
        denominator += dictionary[len(left) - 1][" ".join(left)] * dictionary[len(right) - 1][" ".join(right)]
        
    F *= denominator
    return numerator / F

In [0]:
def dice(words, dictionary):
    words_arr = words.split()
    
    numerator = dictionary[len(words_arr) - 1][words] * 2
    F = 1 / (len(words_arr) - 1)
    denominator = 0
    
    for i in range(len(words_arr) - 1):
        left = words_arr[0:i+1]
        right = words_arr[i+1:]
        
        denominator += dictionary[len(left) - 1][" ".join(left)] + dictionary[len(right) - 1][" ".join(right)]
        
    F *= denominator
    return numerator / F

In [0]:
def LocalMax(text, table):
    xys = dict()
    
#     for nr in tqdm(range(len(text))):
    for nr in range(len(text)):
        line = text[nr]

        text_arr = line.split()

        if len(text_arr) < 2:
            continue

        for i in range(len(text_arr) - 2):
            cur_gram = " ".join(text_arr[i:i+2])

            j = i + 3
            n_gram = 1

            while j < len(text_arr) and n_gram < 6:
                next_gram = " ".join(text_arr[i:j])

                try:
                    xys[cur_gram][1] = max(xys[cur_gram][1], table[next_gram])
                    
                except KeyError:
                    if len(cur_gram.split()) > 2:
                        x = table[" ".join(cur_gram.split()[:-1])]
                        x = max(x, table[" ".join(cur_gram.split()[1:])])
                        xys[cur_gram] = [x, table[next_gram]]
                    else:
                        xys[cur_gram] = [-1, table[next_gram]]

                cur_gram = next_gram

                j += 1
                n_gram += 1

            cur_gram = " ".join(text_arr[i:i+2])
            j = i - 1
            n_gram = 1

            while j >= 0 and n_gram < 6:
                next_gram = " ".join(text_arr[j:i+2])


                try:                        
                    xys[cur_gram][1] = max(xys[cur_gram][1], table[next_gram])
                    
                except KeyError:
                    if len(cur_gram.split()) > 2:
                        x = table[" ".join(cur_gram.split()[:-1])]
                        x = max(x, table[" ".join(cur_gram.split()[1:])])
                        xys[cur_gram] = [x, table[next_gram]]
                    else:
                        xys[cur_gram] = [-1, table[next_gram]]


                cur_gram = next_gram

                j -= 1
                n_gram += 1
              
    return xys

In [0]:
def main(text, max_gram, threshold, glue_fun, verbose=False):
    length = 0
    for line in text:
        length += len(line.split())
    
    if verbose:
        print("Preprocessing text")
    text = preprocessing(text)
    
    if verbose:
        print("Building dictionary with frequencies")

    dictionary = build_dict(text, max_gram)
    
    table = dict()
    
    if verbose:
        print("Proceeding to calculating glue value")
        print("Glue value calculated for:")

    for i in range(2, max_gram + 1):
        if verbose:
            print(i, "grams")
            
        for s in dictionary[i - 1]:
            table[s] = glue_fun(s, dictionary)
    
    if verbose:
        print("Proceeding to Localmax")
    
    RE = dict()
    xys = LocalMax(text, table)
            
    for g in xys:
        if xys[g][0] == -1:
            xys[g][0] = xys[g][1]
        
        val = (xys[g][0] + xys[g][1]) / 2
        
        if table[g] > val and re.match(r"^[a-z\'A-Z\s]*$", g) and dictionary[len(g.split()) - 1][g] > threshold:
            RE[g] = dictionary[len(g.split()) - 1][g] / (length / len(g.split()))
    
    return RE, dictionary

This time, the list of obtainted Relevant Expressions that is returned contains also the term frequency (TF) that will be used when extracting Explict Keywords from documents. The dictionary which contains frequencies of each n-gram (up to 7-gram) is returned as well. It will be useful when calculating the IDF value.

In [0]:
test = text[0]

RE, dictionary = main([test], 7, 1, SCP_f)
print("Obtained", len(RE), "Relevant Expressions")
RE

Obtained 2 Relevant Expressions


{'John Philoponus': 0.0380952380952381, 'in the': 0.0761904761904762}

Effect of applying method to the first paragraph of the corpus. As we can see, the TF value for 'in the' gram is higher than 'John Philoponus' however, we will notice later that taking into account also IDF value will discriminate such irrelevant expressions as 'in the' while extracting Explicit Keywords.

In [0]:
def split_docs(whole_text):
    docs1 = whole_text.split('</doc')
#     docs = []
#     for d in docs1:
#         docs.append(d.split('<doc'))
        
#     return [item for sublist in docs for item in sublist]
    return docs1

In [0]:
import math

docs = split_docs(whole_text)
# n = len(docs)
n = 1000
REs = []
word_count = dict()

for i in tqdm(range(n)):
    RE, dictionary = main(docs[i].split('\n'), 7, 1, SCP_f)
    REs.append(RE)
    
    for grams in dictionary:
        for g in grams:
            try:
                word_count[g] += 1
            except KeyError:
                word_count[g] = 1

/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm_notebook.py:88: TqdmExperimentalWarning: Detect Google Colab 0.0.1a2 and thus load dummy ipywidgets package. Note that UI is different from that in Jupyter. See https://github.com/tqdm/tqdm/pull/640
  " See https://github.com/tqdm/tqdm/pull/640".format(colab.__version__), TqdmExperimentalWarning)


In [0]:
ex_keywords = []

for i in range(len(REs)):
    ex_keywords.append(dict())
    for r in REs[i]:
        tf = REs[i][r]
        idf = math.log10(n / float(word_count[r]))
        ex_keywords[-1][r] = tf * idf

## Results  
Below the results of our method for the first 10 documents were presented. For each document a list of 10-15 most informative Relevant Expressions considered as the Explicit Keywords of the document were presented. They correspond to 10-15 Relevant Expressions having the highest TF-IDF value.

In [0]:
print("--------------------------------------------------")
print("TOP 15 Explicit Keywords for 10 documents")
print("--------------------------------------------------\n")

for i, l in enumerate(ex_keywords[:10]):
    print("Doc", i+1)
    print("--------------------------------------------------")

    for elem, val in sorted(l.items(), key=lambda kv: -kv[1])[:15]:
        print(elem, ":\t", val)
    print('\n')

--------------------------------------------------
TOP 15 Explicit Keywords for documents
--------------------------------------------------

Doc 1
--------------------------------------------------
may be given to any :	 0.00577478344562079
depart for the Holy Land :	 0.00577478344562079
removal of a hydrogen ion :	 0.00577478344562079
Indic scripts :	 0.0046198267564966315
head of a monastery :	 0.0046198267564966315
depart for the Holy :	 0.0046198267564966315
below the national average :	 0.0046198267564966315
removal of a hydrogen :	 0.0046198267564966315
hydrogen ion :	 0.0046198267564966315
a monastery :	 0.004156257947004456
may be given :	 0.0034648700673724736
murder of Naboth :	 0.0034648700673724736
Peter of Courtenay :	 0.0034648700673724736
below the national :	 0.0034648700673724736
Louis the Child :	 0.0034648700673724736


Doc 2
--------------------------------------------------
by a cannonball :	 0.021201413427561835
an independent record :	 0.021201413427561835
footb

Without a shadow of a doubt, not every expression was classified correctly as Explicit Keyword. However, we can also notice quite a satisfying number of expressions that could possibly be considered as Explicit Keywords of the document, like for instance, 'Liberty Cup' or 'Newton's First Law'.

In [0]:
word_doc = dict()

for i in range(len(REs)):
    for w in list(REs[i].keys()):
        try:
            word_doc[w].add(i)
        except KeyError:
            word_doc[w] = set()
            word_doc[w].add(i)

In [0]:
im_keywords = dict()

with tqdm(total=len(word_doc)) as pbar:
    for w1 in word_doc:
        for w2 in word_doc:
            if w1 != w2:
                try:
                    count = len(set.intersection(word_doc[w1], word_doc[w2]))
                    im_keywords[w1].append([w2, count])
                except KeyError:
                    im_keywords[w1] = []
                    im_keywords[w1].append([w2, count])

                try:
                    im_keywords[w2].append([w1, count])
                except KeyError:
                    im_keywords[w2] = []
                    im_keywords[w2].append([w1, count])
                    
        pbar.update(1)

In [0]:
sorted(im_keywords['in the'], key=lambda kv: -kv[1])[:5]

[['of the', 33],
 ['of the', 33],
 ['United States', 13],
 ['United States', 13],
 ['such as', 7]]

In [0]:
print("--------------------------------------------------")
print("TOP Implicit Keywords for document 1")
print("--------------------------------------------------\n")

top0 = sorted(ex_keywords[100].items(), key=lambda kv: -kv[1])[:15]

for k, v in top0:
    print(k, v)
    print(sorted(im_keywords[k], key=lambda kv: -kv[1])[:5])

--------------------------------------------------
TOP Implicit Keywords for document 1
--------------------------------------------------

over the speed of the tractor 0.02737642585551331
[['Great Bedwyn', 1], ['Kennet and Avon', 1], ['Kennet and Avon Canal', 1], ['Avon Canal', 1], ['they look', 1]]
Kennet and Avon Canal 0.018250950570342206
[['Great Bedwyn', 1], ['Kennet and Avon', 1], ['Great Bedwyn', 1], ['Kennet and Avon', 1], ['Avon Canal', 1]]
control over the speed 0.018250950570342206
[['Great Bedwyn', 1], ['Kennet and Avon', 1], ['Kennet and Avon Canal', 1], ['Avon Canal', 1], ['they look', 1]]
speed of the tractor 0.018250950570342206
[['Great Bedwyn', 1], ['Kennet and Avon', 1], ['Kennet and Avon Canal', 1], ['Avon Canal', 1], ['they look', 1]]
Kennet and Avon 0.013688212927756654
[['Great Bedwyn', 1], ['Great Bedwyn', 1], ['Kennet and Avon Canal', 1], ['Avon Canal', 1], ['they look', 1]]
they look at 0.013688212927756654
[['Great Bedwyn', 1], ['Kennet and Avon', 1], ['Ken